# Converting Yelp Dataset json files to CSV


In [ ]:
import pandas as pd
pd.__version__

'1.5.3'

## To match the review texts with business info, we need to merge the two json files

### Load and Clean Business.json file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set up your local path
business_json_path = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'
business = pd.read_json(business_json_path, lines=True)
print(business.shape)
business.head()

(150346, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
# There are significant amount of businesses that are not open anymore
business.is_open.value_counts()

1    119698
0     30648
Name: is_open, dtype: int64

In [ ]:
# Keep only business that are still open
# Drop columns that may not be relavent
df_business = business[business['is_open']==1].drop(['hours','is_open','review_count'], axis=1)
print(df_business.shape)
df_business.head()

(119698, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping..."


### Find relevant categories

One way to split the reviews is by category

In [ ]:
# df.explode requires pandas ver 0.25
# Create one row for each series that contain comma-separated items
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.sample(3)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
146186,qO6gH-xJlkKK58tnSMs1oQ,Starshine Salon,4327 Main St,Philadelphia,PA,19127,40.025455,-75.222817,5.0,"{'BusinessParking': '{'garage': False, 'street...",Hair Stylists
57536,zFcE7bQ2oDfwpPgl_PxOTA,Oystercatchers,2900 Bayport Dr,Tampa,FL,33607,27.962275,-82.549347,4.0,"{'RestaurantsReservations': 'True', 'BusinessP...",American (Traditional)
110706,H6Xs7cFxUe-WfINaL4mVyQ,El Meson Mexican Restaurant,7235 N Keystone Ave,Indianapolis,IN,46240,39.886460,-86.120698,4.0,"{'WiFi': ''free'', 'GoodForKids': 'True', 'Res...",Mexican


In [ ]:
print('Total number of categories:', len(df_explode.categories.value_counts()))
print('Top 10 categories:')
df_explode.categories.value_counts()[:10]

Total number of categories: 1302
Top 10 categories:


Restaurants                  34987
Food                         20419
Shopping                     20186
Home Services                13322
Beauty & Spas                12263
Health & Medical             11046
Local Services               10138
Automotive                    9879
Nightlife                     8379
Event Planning & Services     8173
Name: categories, dtype: int64

In [ ]:
# Exploring Restaurants Category only
df_explode[df_explode['categories'].str.contains('Restaurants', case=True, na=False)].categories.value_counts()

Restaurants           34987
Pop-Up Restaurants       52
Name: categories, dtype: int64

In [ ]:
# Keep only business with categories that are Restaurant related
business_r = df_business[df_business['categories'].str.contains(
                                                                  'Restaurants| Pop-Up Restaurants', case=False, na=False)]
print(business_r.shape)
business_r.head()

(34987, 11)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks"
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B..."


### Loading Massive file as chunks in Pandas

With a huge file like yelp dataset, loading all the data at once will most likely crash your computer.

Luckily, pandas has an option to load large data by segmenting the file into smaller chunks.

In [ ]:
# Set up your local path
review_json_path = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json'

In [ ]:
import pandas as pd

# Set chunk size (smaller if dataset is smaller)
# 2019 Yelp review.json has more than 6 million reviews(rows)
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

### Merge Review.json and Business.json file

By merging only the relevant businesses to the review file, the final dataset will only consist of reviews from those businesses

In [ ]:
# There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_r, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

540141 out of 1,000,000 related reviews
549947 out of 1,000,000 related reviews
524365 out of 1,000,000 related reviews
529735 out of 1,000,000 related reviews
546958 out of 1,000,000 related reviews
544052 out of 1,000,000 related reviews
538572 out of 1,000,000 related reviews
(3773770, 18)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,useful,funny,cool,text,date
2719006,8Wi6ExrCwlUtnhXk7at95A,Santa Fe Cattle Company,2520 Music Valley Dr,Nashville,TN,37214,36.225562,-86.696235,3.5,"{'RestaurantsTakeOut': 'True', 'RestaurantsGoo...","Restaurants, Southern, Steakhouses",dXzNll_yL4xED37ciURvqA,5,0,0,0,We went to the Opryland hotel to see the holid...,2017-12-09 22:09:56
2813519,mpeLyxsDTm5RqZkFT--SdQ,P.F. Chang's,8601 Keystone Xing,Indianapolis,IN,46240,39.912195,-86.108544,3.0,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","Asian Fusion, Restaurants, Chinese, Gluten-Free",TeYB6hVPAFpUOxvqOg7aCg,1,0,0,0,Terrible. It tasted like some cheap Happy Wok!...,2021-04-26 01:59:35
2825242,ghsaIqQ1xY3cWtWWJ4QW5w,Old Granite Street Eatery,243 S Sierra St,Reno,NV,89501,39.522533,-119.813052,4.0,"{'Alcohol': 'u'full_bar'', 'RestaurantsGoodFor...","American (New), American (Traditional), Restau...",0F7N6eBjtXq-1_Z9ZXxBfw,4,0,0,0,"A stylish, rustic looking eatery. The wait was...",2021-12-05 02:56:54


### Convert the new dataframe into a CSV file

By doing so, the smaller dataset can be loaded and shared much more readily

In [ ]:
df.to_csv("yelp_reviews_restaurant_category.csv", index=False)